YOLOS for Fashion
====


In [1]:
from PIL import Image
import supervision as sv
import matplotlib.pyplot as plt
import torch
from transformers import YolosFeatureExtractor, YolosForObjectDetection


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_extractor = YolosFeatureExtractor.from_pretrained(
    'hustvl/yolos-small'
)
model = YolosForObjectDetection.from_pretrained(
    "valentinafeve/yolos-fashionpedia"
).eval().to(device);
